# Linear Convergence of iterations viewed through Fixed-Point Iteration

In this section, we will examine an iteration on estimating the value of a function's "fixed points" in order to examine issues surrounding convergence of an iteration, depending on how we phrase the problem.


> **NOTE:** Find a way to use $g(x) = x - \tan(x)$ in this notebook, use convergence and divergence plots to illustrate where Fixed Point Iterations do or don't work (convergence, error, etc.)

## Setting the Scene

Consider a function $g(x) = x - \cos(x)$. 

If we needed to obtain the root $g(r) = 0$, i.e., $r \in \mathbb{R}:\ 0 = r - \cos(r)$, we might run into issues with finite-precision machine arithmetic which would arise from the subtraction and division of small numbers which could occur in the problem (imagine if it was $x - \tan(x) = x - \frac{\sin(x)}{\cos(x)}$ instead).

So, rephrasing the root finding problem, we instead are trying to solve 

$$ r = \cos(r) $$

Where $r$ is considered a "fixed point" of the tangent function. Abstractly defined,

> **Dfn. Fixed Points** <br> A number $r \in \mathbb{R} : r = g(r)$ is a fixed point for the function $g(x)$.

If we take an initial guess for $r$, we can compute $g(r)$ and measure the difference of it from our guess, using an iteration on $g$ (which in our case, is $\cos(x)$). 

> **Dfn. Fixed Point Iteration** <br>
> Using $x_0$ as an initial guess, for $i = 1, 2, \ldots, n$ iterations, $$x_{i+1} = g(x_i)$$ 
That is, the next guess $x_i$ for each iteration is the value of our chosen function $g$ at our current guess. 

Let's look at it as a code example, and then run it on $g(x) = x - \cos(x)$ to find its roots using the iteration $x_{i+1} = \cos(x_i)$.

In [2]:
import numpy as np

fx = lambda x: np.cos(x)

def fpi(func, initial_guess, max_iter=16, err_tol=1e-08):
    iter_count = 1
    guesses = np.array([initial_guess], dtype=np.float64)
    
    curr_err = err_tol * 10**2
    while iter_count <= max_iter and curr_err >= err_tol:
        last_guess = guesses[-1]
        new_guess = func(last_guess)
        curr_err = np.fabs(new_guess - last_guess)
        
        guesses = np.append(guesses, new_guess)
     
    return guesses
    
        
res = fpi(fx, np.pi)
print(res)

[ 3.14159265 -1.          0.54030231  0.85755322  0.65428979  0.79348036
  0.70136877  0.76395968  0.72210243  0.75041776  0.73140404  0.74423735
  0.73560474  0.74142509  0.73750689  0.74014734  0.7383692   0.7395672
  0.73876032  0.73930389  0.73893776  0.7391844   0.73901826  0.73913018
  0.73905479  0.73910557  0.73907137  0.73909441  0.73907889  0.73908934
  0.7390823   0.73908704  0.73908385  0.739086    0.73908455  0.73908553
  0.73908487  0.73908531  0.73908501  0.73908521  0.73908508  0.73908517
  0.73908511  0.73908515  0.73908512  0.73908514  0.73908513  0.73908514]


However, we'll find that iteration on a problem to find its fixed points will depend on our inital guess and how we defined the function for the fixed point iteration; which we will now examine to begin illustrating the concepts of convergence and error that form a foundation for the methods presented in this course. 

## What?

### Convergence

When we get increasingly close to an answer, we're "converging". When our operation progressively gets farther from an answer to our iteration, it "diverges".

If we let our error for each iterate be denoted as $e_i$, we can start examining what it means for us to find an answer or not (convergence of the iteration). 

We can say that our iteration is __linearly convergent__ if the the ratio of the error at each iterate to the previous error is less than 1. Notationally,

$$ \lim_{i \to \infty} \frac{e_{i+1}}{e_i} = S < 1$$

Generally, we can express convergence with a rate M using 

$$ \lim_{i \to \infty} \frac{e_{i+1}}{(e_i)^q} = M < \infty$$

where $q$ then denotes the _order_ of the convergence. For example, if $q=2$, then the convergence is _quadratic_ (the error at the current step is roughly the square of the next iterate's error, and their ratio exists as a finite number. We'll see this again later with the Newton-Raphson method, another iteration technique. 

For iterations of this type, it's clearly important to know if it will converge or not. If the algorithm diverges under certain conditions, then we're not going to find the answer we're looking for. 


---

That is why we'll add a definition of **locally convergent**. Sometimes, an iteration will only converge if its initial guess is already very close to the answer. 

To keep things compartmentalized, we'll present a theorem representing this here, and then return to the proof of it in the "Why?" section. 

> **Theorem** <br>
> Assume that
* $g$ is a continuously differentiable function,
* $g(r) = r$,
* a constant $S = |g'(r)| < 1$
* Interval of convergence $(r - \epsilon, r + \epsilon)$, where $\epsilon$ is sufficiently small such that $\epsilon$ is less than or equal to the radius of convergence from Calculus, that we may have seen before in the differential equations course.
>
> Then Fixed-Point Iteration converges linearly with rate $S$ to the fixed point $r$ for initial guess $r \pm \epsilon$
> <br> <br>
> Algorithms displaying this error behavior are **locally convergent** for a radius $\epsilon$ around $r$.

### Error Tolerance

A measure of our **precision**, or **absolute error**, at each step will be the absolute value of the difference of our estimate from the truth,

$$ e_{i+1,\ abs} = | x_{i+1} - r | $$

For a fixed point iteration, the difference between successive guesses is our error (since we're either converging or diverging from a root). 

For FPI, we can then see that the absolute error is given by 

$$ e_{i+1} = | x_{i+1} - x_i | $$


A measure of **accuracy**, or **relative error**, at each step for a root $r$ will be 

$$ e_{i,\ relative} = \frac{|x_{i} - r|}{|x_i|} $$

For FPI, the relative error is then

$$ e_{i+1,\ rel} = \frac{|x_{i+1} - x_i|}{|x_{i+1}|} $$

### Stopping Critera

The algorithms that we're looking at could potentially crunch the numbers forever. The set of real numbers is infinite, after all! Furthermore, it can be difficult with variants of a fixed point iteration to determine "in-situ" if we are actually converging or diverging for any given iterate. 

So, we need some basic criteria when designing and implementing iterative algorithms, that define when we want the iteration to stop based on whether an answer has been found or not. 




---

We can specify the significant digits of precision (absolute error) or accuracy (relative error) for the algorithm, and provide that as our "error tolerance". Once our iteration has an error at each step that is less than the tolerance, we know we have converged towards an answer with enough significant digits. 

What becomes important is if we are using a measure of absolute vs relative error. In different instances, high precision may not be possible but a usage of high accuracy (low relative error) may be the best for measuring our approximation. 


---

The other metric we can use is knowledge about our error tolerance and how much our uncertainty is reduced at each step of the iteration. 

Recall that with the Bisection Method, we could analytically determine prior to the computation how many iterations we would need to find an answer within precision (absolute error tolerance).

The idea is to provide a maximum number of iterations that the algorithm will terminate after, and we can look at the history of the guesses to see if we were converging or diverging. We'll examine the tools for this more in depth in the following "Why?" and "How?".

## Why?

### Cobweb Diagrams

### Local Convergence for Fixed-Point Iterations

### Proof of Theorem for Linear, Local Convergence using the Mean-Value Theorem 

## How?

### Rephrasing a Fixed-Point Iteration for Stability, Speed, or Lack Thereof 

## When?


## Where?
